In [1]:
#https://www.oreilly.com/library/view/ai-and-machine/9781492078180/ch04.html


import tensorflow as tf
import tensorflow_datasets as tfds

buffer_size = 10000
batch_size = 2048
#zero = 1.3 Gb
#128 = 2Gb 50%, init = 3 min
#512 = 3.5 Gb 65%, init = 1 min
#2048 = 7.6 Gb 72%, init = 1 min

#horses_or_humans
#patch_camelyon

df, info = tfds.load('patch_camelyon', with_info = True, as_supervised=True)


train_data = df['train']
valid_data = df['validation']
test_data = df['test']
 

2022-10-10 22:35:59.305331: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-10 22:36:00.234448: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libdirectml.0de2b4431c6572ee74152a7ee0cd3fb1534e4a95.so
2022-10-10 22:36:00.234547: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libdxcore.so
2022-10-10 22:36:00.235345: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libd3d12.so
2022-10-10 22:36:00.443642: I tensorflow/c/logging.cc:34] DirectML device enumeration: found 1 compatible adapters.
/home/lachlan/.pyenv/versions/venv_patchcam/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
#A function to help scale the images
def preprocess(image, labels):
  image = tf.cast(image, tf.float32)
  image /= 255.
  return image, labels

#Applying the preprocess function we the use of map() method
train_data = train_data.map(preprocess)
valid_data = valid_data.map(preprocess)
test_data = test_data.map(preprocess)

train_batches = train_data.shuffle(buffer_size).batch(batch_size).prefetch(1)
valid_data = valid_data.shuffle(buffer_size).batch(batch_size).prefetch(1)
test_data = test_data.shuffle(buffer_size).batch(batch_size).prefetch(1)

In [3]:


model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', 
    #                       input_shape=(300, 300, 3)),
                            input_shape=(96, 96, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='Adam', loss='binary_crossentropy',
metrics=['accuracy'])

In [4]:
if False:
    print(train_batches)
    print(train_batches.cardinality().numpy())

    next(iter(train_batches))

In [5]:
history = model.fit(train_batches, epochs=5, validation_data = valid_data, verbose=2)

print("FINISHED")

Epoch 1/5


2022-10-10 22:36:02.152529: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-10 22:36:02.183250: W tensorflow/core/grappler/utils/graph_view.cc:849] No registered 'DataFormatVecPermute' OpKernel for GPU devices compatible with node {{node gradient_tape/sequential/conv2d_4/Conv2D/Conv2DBackpropInput-0-DataFormatVecPermuteNHWCToNCHW-LayoutOptimizer}}
	.  Registered:  device='GPU'; T in [DT_INT64]
  device='GPU'; T in [DT_INT32]
  device='CPU'; label='host'; T in [DT_INT64]
  device='CPU'; label='host'; T in [DT_INT32]
  device='CPU'; T in [DT_INT64]
  device='CPU'; T in [DT_INT32]

2022-10-10 22:36:02.184702: W tensorflow/core/grappler/utils/graph_view.cc:849] No registered 'DataFormatVecPermute' OpKernel for GPU devices compatible with node {{node gradient_tape/sequential/conv2d_3/Conv2D/Conv2DBackpropInput-0-DataFormatVecPermuteNHWCToNCHW-LayoutOptimizer}}
	.  Registered:  device='GPU'; T in [DT_INT64

128/128 - 47s - loss: 0.5154 - accuracy: 0.7447 - val_loss: 0.4866 - val_accuracy: 0.7773 - 47s/epoch - 371ms/step
Epoch 2/5
128/128 - 41s - loss: 0.4379 - accuracy: 0.8035 - val_loss: 0.4738 - val_accuracy: 0.7752 - 41s/epoch - 323ms/step
Epoch 3/5
128/128 - 39s - loss: 0.3988 - accuracy: 0.8245 - val_loss: 0.5342 - val_accuracy: 0.7523 - 39s/epoch - 303ms/step
Epoch 4/5
128/128 - 39s - loss: 0.3583 - accuracy: 0.8438 - val_loss: 0.4938 - val_accuracy: 0.8034 - 39s/epoch - 302ms/step
Epoch 5/5
128/128 - 40s - loss: 0.3362 - accuracy: 0.8549 - val_loss: 0.4671 - val_accuracy: 0.8127 - 40s/epoch - 312ms/step
FINISHED
